In [1]:
from typing import TYPE_CHECKING


if TYPE_CHECKING:
    from math_rag.application.containers import ApplicationContainer
    from math_rag.infrastructure.containers import InfrastructureContainer

    application_container: ApplicationContainer
    infrastructure_container: InfrastructureContainer

In [2]:
RESET = False
%load_ext hooks.notebook_hook

2025-07-15 23:24:32,870 - INFO - datasets - config.py:54 - PyTorch version 2.6.0 available.


### Playground

In [ ]:
from agents import Agent, Runner


math_problem_solver_agent = Agent(
    name='Assistant',
    instructions='You are a helpful assistant',
    output_type=str,
)

result = await Runner.run(
    math_problem_solver_agent, 'Write a haiku about recursion in programming.'
)
print(result.final_output)

Code repeats itself,  
Functions within functions loop—  
Infinite echoes.


In [ ]:
TODO = ...

math_problem_solver_agent = Agent(
    name=TODO,
    instructions=TODO,
    prompt=TODO,
    handoff_description=TODO,
    handoffs=TODO,
    model=TODO,
    model_settings=TODO,
    tools=TODO,
    mcp_servers=TODO,
    mcp_config=TODO,
    input_guardrails=TODO,
    output_guardrails=TODO,
    output_type=TODO,
    hooks=TODO,
    tool_use_behavior=TODO,
    reset_tool_choice=TODO,
)

In [ ]:
# TODO use openai guardrail model

In [ ]:
from agents import Handoff
from agents.extensions.handoff_filters import remove_all_tools


tool: FunctionTool = ...
math_problem_solver_agent: Agent = ...

handoff = Handoff(
    tool_name=tool.name,
    tool_description=tool.description,
    input_json_schema=tool.params_json_schema,
    on_invoke_handoff=tool.on_invoke_tool,
    agent_name=math_problem_solver_agent.name,
    input_filter=remove_all_tools,
    strict_json_schema=True,
    is_enabled=True,
)

In [ ]:
from agents import HandoffInputData


def drop_pre_handoff_items(data: HandoffInputData) -> HandoffInputData:
    """
    Remove everything generated before the handoff turn,
    so the next agent only sees the original history + new_items.
    """
    return HandoffInputData(
        input_history=data.input_history,
        pre_handoff_items=(),
        new_items=data.new_items,
    )


def keep_last_5_new_items(data: HandoffInputData) -> HandoffInputData:
    """
    Keep only the last 5 items from new_items,
    preserving full history and pre_handoff_items.
    """
    return HandoffInputData(
        input_history=data.input_history,
        pre_handoff_items=data.pre_handoff_items,
        new_items=tuple(data.new_items[-5:]),
    )

### Jupyter tool

In [3]:
jupyter_client = infrastructure_container.jupyter_client()

In [4]:
await jupyter_client.start_session(user_id='0')

JupyterStartSessionResult(message='Session started successfully', notebook_path='/mnt/jupyter_sessions/0/notebook_0.ipynb')

In [ ]:
from agents import Agent, Runner


math_problem_solver_agent = Agent(
    name='CodeRunner',
    instructions='Use the execute_code tool to run Python code snippets.',
    tools=[execute_code_tool],
)

result = await Runner.run(
    math_problem_solver_agent, 'Please run some Python code, invent it on your own'
)
result.final_output

'Here are 10 random integers between 1 and 100:\n\\[69, 74, 94, 5, 33, 70, 18, 40, 26, 50\\]'

In [9]:
for x in result.new_items:
    if hasattr(x, 'output'):
        print(x.output)

{"output":"[69, 74, 94, 5, 33, 70, 18, 40, 26, 50]","error":null}


In [ ]:
await jupyter_client.end_session(user_id='0')

### Katex tool

In [ ]:
from agents import Agent, Runner


math_problem_solver_agent = Agent(
    name='CodeRunner',
    instructions='Use the validate_katex tool to validate KaTeX.',
    tools=[validate_katex_tool],
)

result = await Runner.run(
    math_problem_solver_agent, 'Please validate some KaTeX, invent it on your own'
)
result.final_output

'The KaTeX expression \\( c = \\pm\\sqrt{a^2 + b^2} \\) is valid.'

### Graph tool

In [ ]:
from agents import Agent, Runner


math_problem_solver_agent = Agent(
    name='CodeRunner',
    instructions='Use the search_graph tool to search the knowledge graph.',
    tools=[search_graph_tool],
)

result = await Runner.run(
    math_problem_solver_agent, 'Please search graph, send a query about logistic regression'
)
result.final_output

"Here are some details about logistic regression derived from the search graph:\n\n1. **Function Minimization**:\n   - The function \\( f(\\mathbf{x}) \\) is minimized using methods like Newton's method, where the function is evaluated at the current point in the parameter space.\n   - The quadratic approximation at a point is derived from the Taylor series expansion around that point.\n\n2. **Taylor Series Expansion**:\n   - For functions of a single variable, like \\( f(x) \\), the function's quadratic approximation is derived using its Taylor series.\n   - The point \\( a \\) serves as the center of the expansion, from which the function is approximated.\n\n3. **Initial Point in Optimization**:\n   - The symbol \\( x_{0} \\) denotes the initial point for starting the minimization process using Newton's method.\n\n4. **Approximation of Functions**:\n   - The Taylor series provides a way to approximate functions at a point.\n   - Specifically, the second-order Taylor expansion offers 

### Web search tool

In [5]:
from agents import Agent, Runner, WebSearchTool


math_problem_solver_agent = Agent(
    name='CodeRunner',
    instructions='Use the web search tool.',
    tools=[WebSearchTool(search_context_size='low')],
)

result = await Runner.run(math_problem_solver_agent, 'search for the Pythagorean theorem')
result.final_output

'The Pythagorean theorem is a fundamental principle in geometry that describes the relationship between the sides of a right-angled triangle. It states that the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides. This relationship is expressed algebraically as:\n\n\\[ a^2 + b^2 = c^2 \\]\n\nwhere \\( a \\) and \\( b \\) are the lengths of the legs (the sides forming the right angle), and \\( c \\) is the length of the hypotenuse.\n\n**Example:**\n\nConsider a right-angled triangle with legs measuring 3 units and 4 units. To find the length of the hypotenuse:\n\n\\[ 3^2 + 4^2 = c^2 \\]\n\n\\[ 9 + 16 = c^2 \\]\n\n\\[ 25 = c^2 \\]\n\nTaking the square root of both sides:\n\n\\[ c = \\sqrt{25} = 5 \\]\n\nTherefore, the hypotenuse measures 5 units.\n\n**Historical Context:**\n\nAlthough named after the ancient Greek mathematician Pythagoras (c. 570–500/490 BCE), evidence suggests that the relati

## Full Agent

In [3]:
problem_text = """
1. (N) We trained the logistic regression model on a set of labeled examples $\\mathcal{D}$. We obtained some weight vector $\\mathbf{{w}}$ and bias $w_0=0.15$. For the so-obtained model, an example $\\mathbf{{x}}$, whose label is $y=0$, inflicts a cross-entropy loss of $L(0, h(\\mathbf{{x}}))=0.274$. What would be the value of the cross-entropy loss for the example $x$ if we multiplied its features by two and changed its label?
"""

In [4]:
jupyter_client = infrastructure_container.jupyter_client()

await jupyter_client.start_session(user_id='0')

JupyterStartSessionResult(message='Session started successfully', notebook_path='/mnt/jupyter_sessions/0/notebook_0.ipynb')

In [5]:
from functools import partial
from uuid import UUID

from agents import Agent, Runner
from IPython.display import Latex, display

from math_rag.application.agents.tools import (
    execute_code_tool,
    search_graph_tool,
    validate_katex_tool,
)


# configure the search tool with your math expression index
math_expression_index_id = UUID('4916077a-1d6d-4ef1-a552-bb9668721052')
search_graph_tool.on_invoke_tool = partial(
    search_graph_tool.on_invoke_tool,
    math_expression_index_id=math_expression_index_id,
)

# main solver agent
math_problem_solver_agent = Agent(
    name='MathProblemSolverAgent',
    model='o4-mini',
    instructions=(
        'Solve the given math problem step by step. '
        'While solving, search the knowledge graph, write code, and execute it in the provided code sandbox. '
        'The final answer must include formulas and detailed steps, but do not show any code in the final answer. '
        'Once you have the final solution, call the transfer_to_katex_validator_agent tool with no arguments '
        'to validate and correct all KaTeX syntax before returning.'
    ),
    tools=[search_graph_tool, execute_code_tool],
)

input = (
    f'Solve the following problem by using knowledge graph and code sandbox. '
    f'You must use search_graph_tool and execute_code_tool tools.'
    f'return detailed steps at the end: \n {problem_text}'
)
solver_result = await Runner.run(math_problem_solver_agent, input=input)

In [7]:
# KaTeX validator agent
katex_validator_agent = Agent(
    name='KatexValidatorAgent',
    model='o4-mini',
    instructions=(
        'Given the complete solution to the math problem, '
        'identify all KaTeX sections and validate them using the provided tool. '
        'If any issues are found, fix the KaTeX syntax and validate again. '
        'Repeat this process until all KaTeX is valid. '
        'Finally, return the full solution with corrected KaTeX sections. '
        'Do not include KaTeX in code cells; KaTeX must be rendered in the chat interface.'
        'All KaTeX expressions must be bound with "$" symbols.'
    ),
    tools=[validate_katex_tool],
)

input = (
    f'Validate and fix KaTeX sections in given solution by following steps: '
    f'1. identify all math expressions in given text '
    f'2. validate each KaTeX expression using `validate_katex_tool` '
    f'3. check if you found and validate all KaTeX expression '
    f'4. bound each KaTeX expression with single `$` (for inline KaTeX) or double `$$` (for multiline KaTeX)'
    f'5. do NOT add additional explanations like "Here is the detailed solution with all KaTeX expressions properly delimited and validated:"'
    f'Your Task: \n{solver_result.final_output}'
)
validator_result = await Runner.run(katex_validator_agent, input=input, max_turns=50)

display(Latex(validator_result.final_output))

<IPython.core.display.Latex object>

In [10]:
await jupyter_client.end_session(user_id='0')

JupyterEndSessionResult(message='Session ended successfully')

In [ ]:
math_expression_index_searcher_service = (
    await application_container.math_expression_index_searcher_service()
)
math_expression_relationship_serializer_service = (
    application_container.math_expression_relationship_serializer_service()
)
from uuid import UUID


math_expression_index_id = UUID('4916077a-1d6d-4ef1-a552-bb9668721052')

math_expression_relationship_ids = await math_expression_index_searcher_service.search(
    math_expression_index_id,
    query='',
    query_limit=8,
    limit=4,
)
result = await math_expression_relationship_serializer_service.serialize(
    math_expression_relationship_ids
)
result